# IPOTESI DI PRIMA BOZZA PER REGOLA 

Idea metto tutti gli attirbuti possibili di buildings in osm.
E poi ottengo i dataframe per ogni macroclasse con tutti i tipi di edificio e i numeri.
E poi scegliamo magari delle macrogateorie per raggruppare gli edifici secondo una classificazione nostra,che corrisponde poi a quella dei file di ramp. 
Tipo magari nel tagbuildign ci sono 4 tipi di edificio religioso con tag diverso, li riuniamo sotto un'unico tag church a cui corrisponde un file di modello di domanda.

In [ ]:
import pandas as pd

# Lettura del file Excel
building_type = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/number_buildings_type.xlsx"
)

# Dizionario dei tipi di edificio
type_dict = {
    "religious_type": [
        "religious",
        "cathedral",
        "chapel",
        "church",
        "kingdom_hall",
        "monastery",
        "mosque",
        "presbytery",
        "shrine",
        "synagogue",
        "temple",
    ],
    "accomodation_type": [
        "apartments",
        "barracks",
        "bungalow",
        "cabin",
        "detached",
        "dormitory",
        "farm",
        "ger",
        "hotel",
        "house",
        "houseboat",
        "residential",
        "semidetached_house",
        "static_caravan",
        "stilt_house",
        "terrace",
        "tree_house",
        "trullo",
    ],
    "commercial_type": [
        "commercial",
        "industrial",
        "kiosk",
        "office",
        "retail",
        "supermarket",
        "warehouse",
    ],
    "civic_type": [
        "bakehouse",
        "bridge",
        "civic",
        "college",
        "fire_station",
        "government",
        "gatehouse",
        "hospital",
        "kindergarten",
        "museum",
        "public",
        "school",
        "toilets",
        "train_station",
        "transportation",
        "university",
    ],
    "agriculture_type": [
        "barn",
        "conservatory",
        "cowshed",
        "farm_auxiliary",
        "greenhouse",
        "slurry_tank",
        "stable",
        "sty",
        "livestock",
    ],
    "sport_type": [
        "grandstand",
        "pavilion",
        "riding_hall",
        "sports_hall",
        "sports_centre",
        "stadium",
    ],
    "cars_type": ["carport", "garage", "garages", "parking"],
    "technical_type": [
        "digester",
        "service",
        "tech_cab",
        "transformer_tower",
        "water_tower",
        "storage_tank",
        "silo",
    ],
    "other_type": [
        "beach_hut",
        "bunker",
        "castle",
        "construction",
        "container",
        "guardhouse",
        "military",
        "outbuilding",
        "pagoda",
        "quonset_hut",
        "roof",
        "tent",
        "tower",
        "windmill",
        "yes",
        "user defined",
    ],
}

# Creazione dei DataFrame vuoti
dataframes = {}
for key, value in type_dict.items():
    df = pd.DataFrame(index=value)
    # Aggiunta delle colonne per i cluster
    for cluster in building_type["cluster"].unique():
        df[f"cluster_{cluster}"] = 0
    dataframes[key] = df


# Funzione per aggiornare i DataFrame per ogni cluster separatamente
def update_dataframes_by_cluster(main_df, dataframes):
    clusters = main_df["cluster"].unique()

    for cluster in clusters:
        cluster_df = main_df[main_df["cluster"] == cluster]

        for _, row in cluster_df.iterrows():
            for column in main_df.columns[1:]:  # Ignorare la colonna 'cluster'
                value = row[column]
                if value > 0:
                    for df in dataframes.values():
                        if column in df.index:
                            df.at[column, f"cluster_{cluster}"] += value


# Aggiornamento dei DataFrame per ogni cluster separatamente
update_dataframes_by_cluster(building_type, dataframes)

# Concatenazione dei DataFrame in un unico DataFrame per tipo di edificio
concatenated_dfs = {}
for key, df in dataframes.items():
    concatenated_dfs[key] = df
output_path = "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/"
for key, df in concatenated_dfs.items():
    df.to_csv(f"{output_path}_{key}.csv", index=True)

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

cleaned_buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/cluster_with_buildings.geojson"
)
house = cleaned_buildings[cleaned_buildings["tags_building"] == "house"]
area_tot = house["area"].sum()
pop_microgrid = (
    93641.59  # Qua devi usare la funzione nel codice per calcolare la popolazione.
)
density = pop_microgrid / area_tot
grouped_buildings = cleaned_buildings.groupby("cluster_id")
clusters = np.sort(cleaned_buildings["cluster_id"].unique())
house_area_for_cluster = []
for cluster in clusters:
    cluster_buildings = pd.DataFrame(grouped_buildings.get_group(cluster))
    house = cluster_buildings[cluster_buildings["tags_building"] == "house"]
    area_house = house["area"].sum()
    house_area_for_cluster.append(area_house)
population_df = pd.DataFrame()
population_df["cluster"] = clusters
population_df.set_index("cluster", inplace=True)
population_df["house_area_for_cluster"] = house_area_for_cluster
people_for_cluster = (population_df["house_area_for_cluster"] * density).round()
population_df["people_for_cluster"] = people_for_cluster

display(population_df)

In [ ]:
tier_percent = [0.1, 0.2, 0.3, 0.2, 0.1, 0.1]
people_for_cluster = population_df["people_for_cluster"]
for i in tier_percent:
    people_in_tier = people_for_cluster * i
    population_df[f"people_for_tier{i}"] = people_in_tier

display(population_df)

# CREAZIONE DEL FILE ALL_RAW_BUILDINGS PER IL TUTORIAL IN NIGERIA

In [ ]:
import osmium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, Polygon


class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.nodes = {}
        self.ways = []
        self.buildings = []

    def node(self, n):
        if n.location.valid():
            self.nodes[n.id] = (n.location.lon, n.location.lat)

    def way(self, w):
        if "building" in w.tags:
            nodes = []
            for node in w.nodes:
                if node.ref in self.nodes:
                    nodes.append(self.nodes[node.ref])
            if nodes:
                tags = {tag.k: tag.v for tag in w.tags if tag.k.startswith("building")}
                self.buildings.append([w.id, Polygon(nodes), tags])
        else:
            nodes = []
            for node in w.nodes:
                if node.ref in self.nodes:
                    nodes.append(self.nodes[node.ref])
            if nodes:
                self.ways.append([w.id, LineString(nodes)])


# Specifica il percorso al tuo file .osm
osm_file_path = "C:/Users/margh/Desktop/pypsa/pypsa-distribution/map.osm"

# Crea un handler per leggere il file OSM
handler = OSMHandler()

# Applica l'handler al file OSM
handler.apply_file(osm_file_path)

# Converti i nodi in un GeoDataFrame
nodes_gdf = gpd.GeoDataFrame(
    {
        "id": list(handler.nodes.keys()),
        "geometry": [Point(coord) for coord in handler.nodes.values()],
    }
)

# Converti le strade (ways) in un GeoDataFrame
ways_gdf = gpd.GeoDataFrame(handler.ways, columns=["id", "geometry"])

# Converti gli edifici in un GeoDataFrame
buildings_gdf = gpd.GeoDataFrame(handler.buildings, columns=["id", "geometry", "tags"])

# Aggiungi i tag come colonne separate nel GeoDataFrame
tags_df = buildings_gdf["tags"].apply(pd.Series)
buildings_gdf = buildings_gdf.join(tags_df).drop(columns=["tags"])
buildings_gdf.rename(columns={"building": "tags.building"}, inplace=True)
# Stampa i GeoDataFrame


# Salva il GeoDataFrame degli edifici in un file
buildings_gdf.to_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/osm/raw/all_raw_building.geojson",
    driver="GeoJSON",
)

In [ ]:
buildings_gdf.iloc[:, 2].unique()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os

buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/osm/raw/all_raw_building.geojson"
)
fig, ax = plt.subplots(figsize=(100, 70), subplot_kw={"projection": ccrs.PlateCarree()})
buildings.plot(
    ax=ax,
    legend=False,
    label="buildings",
    color="midnightblue",
    markersize=0.05,
    alpha=1,
)